In [4]:
import pandas as pd
import requests
import transformer
import numpy as np
from time import sleep
import csv
import random
import json

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate, br",
    "cookie": "_T_WM=95312399235; XSRF-TOKEN=3a31a4; WEIBOCN_FROM=1110006030; SCF=AqAA_hXhYzgggeI_jiVgO_sAtey7ebyoB0r2lIfovSCppuC-IMQSaL-eEiD20ySrP_9VisloRCBQLI6yErrA_VI.; SUB=_2A25Lq1k5DeRhGeFP4lMR8SvIyTmIHXVoydTxrDV6PUJbktCOLWnzkW1NQOrKxXaGJKpvyPl0o-n9E8GC9sXW8mxg; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFJf.nPypequXxyIBhJbGay5NHD95QNeK.peh2fShzfWs4DqcjPi--ci-z0i-2Ei--NiKn4i-z41h.Eehzt; SSOLoginState=1722755433; ALF=1725347433; MLOGIN=1; mweibo_short_token=c0319b197f; M_WEIBOCN_PARAMS=lfid%3D231583%26luicode%3D20000174%26uicode%3D20000174",
}

# 高德API 请求地址前缀
req_url_pref = "https://restapi.amap.com/v3/geocode/geo?"
amp_api_key = '61c256acd1b853e34a6b9a31033c399d'
amp_api_city='上海市'
page=1

def GaoDeLocation(address):
        # 没有上海市的地址在前面加上上海市
        if address.find("上海市") == -1:
            address = "上海市" + address
        rep_params={"key":amp_api_key,"address":address,"city":amp_api_city}
        response=requests.get(req_url_pref,params=rep_params)
        locationinfo=response.json()
        status=locationinfo["status"]
        if status !="0":
            outputaddress=locationinfo["geocodes"][0]["formatted_address"]
            gcj02_lng=float(locationinfo["geocodes"][0]["location"].split(",")[0])
            gcj02_lat=float(locationinfo["geocodes"][0]["location"].split(",")[1])
            wgs84_lng,wgs84_lat=transformer.gcj02towgs84(gcj02_lng,gcj02_lat)
            wgs84_lng=float(wgs84_lng)
            wgs84_lat=float(wgs84_lat)
        else:
            outputaddress=np.nan
            gcj02_lng=np.nan
            gcj02_lat=np.nan
            wgs84_lng=np.nan
            wgs84_lat=np.nan

        return outputaddress,gcj02_lng,gcj02_lat,wgs84_lng,wgs84_lat

def write_nameaddress_csv(filename, fields, data):
    with open(filename, mode="a", newline="",encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=fields)
        if file.tell() == 0:
            writer.writeheader()
        writer.writerow(data)

def get_tweets(URL):
    # url = URL.format(str(page))
    url=URL.format(1)
    while True:
        try:
            res = requests.get(url,headers=headers)
            res.encoding = "utf-8-sig"
            # print(res)

            # 间隔时间
            # sleep(random.randint(5, 9))
            sleep(random.randint(2,4))
            # soup = BeautifulSoup(res.text, 'html.parser')

            jd = json.loads(res.text)
            jd = res.json()

        except:
            print("代理有问题呀，换个ip试试")
            continue
        if (jd["ok"] == 0) and ("这里还没有内容" in str(jd)):
            print(jd)
            return 0
        if jd["ok"] == 0:
            print("获取地点的页面失败啊，换个ip试试")
        else:
            break
    # 第一页的结果会有点不一样
    if page == 1:
        # if jd['data']['cards'][0]['card_id']=='card_hq_poiweibo':  #微博数据结构的表头为data——cards—— "card_id": "card_hq_poiweibo",
        if (
            jd["data"]["cards"][0]["card_id"] == "hot_search"
        ):  # 微博数据结构的表头为data——cards—— "card_id": "card_hq_poiweibo",
            tweets = jd["data"]["cards"]
            # tweets=jd['data']['cards'][0]['card_group']
        else:
            tweets = jd["data"]["cards"]  # 爬取city的这个地方我自己改为0了.
            # tweets=jd['data']['cards'][1]['card_group']


        name=jd["data"]["cardlistInfo"]["title_top"]
        address=jd["data"]["cardlistInfo"]["address"]
        subtitle=jd["data"]["cardlistInfo"]["sub_title"]

        # 没有上海市的地址在前面加上上海市
        if address.find("上海市") == -1:
            address = "上海市" + address
        rep_params={"key":amp_api_key,"address":address,"city":amp_api_city}
        response=requests.get(req_url_pref,params=rep_params)
        locationinfo=response.json()
        status=locationinfo["status"]
        if status !="0":
            outputaddress=locationinfo["geocodes"][0]["formatted_address"]
            gcj02_lng=float(locationinfo["geocodes"][0]["location"].split(",")[0])
            gcj02_lat=float(locationinfo["geocodes"][0]["location"].split(",")[1])
            wgs84_lng,wgs84_lat=transformer.gcj02towgs84(gcj02_lng,gcj02_lat)
            wgs84_lng=float(wgs84_lng)
            wgs84_lat=float(wgs84_lat)
        else:
            outputaddress=np.nan
            gcj02_lng=np.nan
            gcj02_lat=np.nan
            wgs84_lng=np.nan
            wgs84_lat=np.nan

        outputaddress,gcj02_lng,gcj02_lat,wgs84_lng,wgs84_lat=GaoDeLocation(address)

        write_nameaddress_csv(
            "poi_info_added.csv",
            ["name", "address","subtitle","outputaddress","gcj02_lng","gcj02_lat","wgs84_lng","wgs84_lat"],
            {
                "name": name,
                "address": address,
                "subtitle":subtitle,
                "outputaddress":outputaddress,
                "gcj02_lng":gcj02_lng,
                "gcj02_lat":gcj02_lat,
                "wgs84_lng":wgs84_lng,
                "wgs84_lat":wgs84_lat
            },
        )
    else:
        tweets = jd["data"]["cards"]
        # tweets=jd['data']['cards'][0]['card_group']

    print("获取到{}条微博".format(len(tweets)))
    return tweets

for row in range(72):
    global conn, cur, place, pid, category, category_id, total_id, city, city_id
    # 读取资料文档
    # f = pd.read_csv('data/pid.csv', encoding='gbk')  # 出错的话改成gbk
    f = pd.read_csv("poi点位置信息补充.csv",encoding='utf-8-sig')  # 出错的话改成gbk
    # f = pd.read_csv("data\shanghai.csv")  # 出错的话改成gbk

    place = f["pname"][row]
    pid = f["pid"][row]
    category = f["category"][row]
    category_id = f["category_id"][row]
    total_id = f["total_id"][row]
    city = f["city"][row]
    city_id = f["city_id"][row]

    URL = "https://m.weibo.cn/api/container/getIndex?containerid=" + pid + "&page={}"
    time.sleep(random.randint(3, 6))
    tweets = get_tweets(URL)





获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
{'ok': 0, 'msg': '这里还没有内容', 'data': {'cards': []}}
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到1条微博
获取到1条微博
获取到1条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到1条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到1条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到1条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到1条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到1条微博
获取到1条微博
获取到1条微博
获取到2条微博
获取到1条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
获取到2条微博
